
#note reasoning for batches is in comparing_outliers.csv
#0 manual processing in interactive R session on cluster 
#1 to process each sample individually via slurm submission - avoiding as it results in different numbers of rows in count matrix of each sample which prevents merging later
#2 to process in bulk up to or including nnsvg step

skip ahead to 2


2. To process all samples together in a slurm submission (either up to nnsvg or inc nnsvg)
#make sure you are updating and using the scratch scripts, there are also copies on mnt but outdated

In [ ]:
#2.0

sbatch /mnt/scratchc/fmlab/lythgo02/visium_data/scripts/nnsvg.sh

#which is 

#!/bin/bash

#SBATCH --error=nnSVG.err
#SBATCH --time=0200:00:00             
#SBATCH --cpus-per-task=32      
#SBATCH --mem=120G                   
#SBATCH --partition=epyc     


# Run the R script
#Rscript /mnt/scratchc/fmlab/lythgo02/visium_data/scripts/sample_specific_filtering_wholeworkflow.r 

In [ ]:
#there is also a script called end_stage_hvg_nnsvg_singlesamplejob.r which is the end section of the workflow to submit single jobs for each sample for hvg and nnsvg

In [ ]:
# 2.1 then process each sample separately for hvg and nnsvg via slurm job using following script 

# note after separating out individual samples, SITSA1 and SITSD3 have rows with 0 count (causing nnsvg to fail) for:
#SITSA1 - HNRNPCL3, TMOD4, REG1A USP17L10, PVRIG CNTNAP3B, ZP1, C13orf46 ZP2, PAGE2, TGIF2LY, PCDH11Y 
#SITSD3 - HNRNPCL3, PRAMEF33, UBE2U, CA14 AL590560.2, WDR64, SPDYA, NMS, ZDHHC19, USP17L10, HTN3, PCBD2, HIST1H4E, PVRIG 
         #ASZ1, AKR1C4, OOSP1, ZP1, DEFB131B, C13orf46, GOLGA6L6, TEPP, ALOX15, ANKRD30B, ZNF541, BPIFB2, BPIFB6, BPIFB3 
         #USP41, CRYBB2, GAGE1, PAGE2, SLC25A5, TGIF2LY, PRKY 
# remove zero expression genes
ix_zero_genes <- rowSums(counts(spe_SITSD3)) == 0
table(ix_zero_genes)

# identify genes with 0 expression
true <- ix_zero_genes[ix_zero_genes]

if (sum(ix_zero_genes) > 0) {
  spe_SITSD3 <- spe_SITSD3[!ix_zero_genes, ]}

# remove spots with zero expression
ix_zero_spots <- colSums(counts(spe_SITSA1)) == 0
table(ix_zero_spots) #0

In [ ]:
#2.1 
for sample in SITSA1 SITSA2 SITSA3 SITSA4 SITSB1 SITSB2 SITSB3 SITSB4 SITSC1 SITSC2 SITSC3 SITSC4 SITSD1 SITSD2 SITSD3 SITSD4 SITSE2 SITSE4 SITSF2 SITSF4 SITSG2 SITSH2; 
do     sbatch ../scripts/nnsvg.sh $sample; done

2.3 then open interactive R session and proceed as follows
We will take the top ranking HVGs and top ranking SVGs, reduce the dimensions with PCA on each, and then integrate and cluster


In [ ]:
srun --pty --job-name=interactive_r --mem=68G --cpus-per-task=4 --partition=epyc --time=100:00:00 bash

micromamba activate r_env 

R

In [ ]:
#2.4 fitting top HVGs

knitr::opts_chunk$set(echo = TRUE)
library(spatialLIBD)
library(nnSVG)
library(SpatialExperiment)  
library(scater)
library(AnnotationHub)
library(tidyverse)
library(ggspavis)
library(scran)
library(DT)
projDir = "/mnt/scratchc/fmlab/lythgo02/visium_data/"



In [ ]:
#load samples
load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/" 

# List files in the directory
files <- list.files(path = load_all_data, pattern = "spe_sub_modelgenevar", full.names = TRUE) #to load hvgs only

#assign names so they are loaded into the spe with the correct identifiers
samplesheet <- c("SITSA1", "SITSA2", "SITSA3", "SITSA4",
                 "SITSB1", "SITSB2", "SITSB3", "SITSB4", 
                 "SITSC1", "SITSC2", "SITSC3", "SITSC4", 
                 "SITSD1", "SITSD2", "SITSD3", "SITSD4",
                 "SITSE2", "SITSE4", "SITSF2", "SITSF4",
                 "SITSG2", "SITSH2")  

names(files) <- samplesheet

# Initialize a list to store each sample
spe_list <- list()

# Loop through each file and process it
for (sample_name in names(files)) {
 # Read the Visium data
  spe <- readRDS(files[sample_name])

  # Store the processed sample in the list
  spe_list[[sample_name]] <- spe 
  
  
  assign(paste0("spe_", sample_name), spe)
  #print progress check
  cat("Loaded:", sample_name, "\n")
}


Dimensionality reduction on top HVGs 
Top HVGs have been determined using getTopHVGs() from scran which returns top 10% (prop=0.1) of genes with highest biological variance after discarding those with bio <= 0

In [ ]:

# Compute PCA for HVGs (highly variable genes)
#I have already run 
 # top_hvgs <- getTopHVGs(dec, prop = 0.1, var.field = "bio")
 #var.field = "bio"	Ranks genes by biological variability
 #prop = 0.1	Takes top 10% of genes based on that ranking

 # and assigned TRUE or FALSE for hvgs
set.seed(123)

#initialise list to store hvgs
hvg <- vector("list", length(spe_list))

for (i in seq_along(spe_list)) {
     #extract hvg list
     hvg[[i]] <- as.data.frame(rowData(spe_list[[i]])) %>%
     filter(hvg==TRUE) %>%
     pull(symbol)

 spe_list[[i]] <- runPCA(spe_list[[i]], subset_row = hvg[[i]])  
}
## Set seed
set.seed(987)
## Compute UMAP on top 50 PCs

for (i in seq_along(spe_list)) {

spe_list[[i]] <- runUMAP(spe_list[[i]], dimred = "PCA") 
}

for (i in seq_along(spe_list)) {

## Check correctness - names
print(reducedDimNames(spe_list[[i]])) 
#check correctness
print(dim(reducedDim(spe_list[[i]], "UMAP")))

## Update column names for easier plotting
colnames(reducedDim(spe_list[[i]], "UMAP")) <- paste0("UMAP", 1:2)
}

plots <- vector("list", length(spe_list))
#plots
for (i in seq_along(spe_list)) {
     #extract sample id
     sample_id <- spe_list[[i]]$sample_id[i]
     
plots[[i]] <- ggplot(data = as.data.frame(spe_list[[i]]@int_colData@listData$reducedDims$PCA),
       aes(x = PC1, y = PC2, colour = spe_list[[i]]@colData$ground_truth)) + 
  geom_point(size = 0.5) + 
  scale_colour_brewer(type = "qual") + 
  labs(title = paste("Reduced dimensions: PCA (Sample:", sample_id, ")"),
       x = "PC1",
       y = "PC2",
       colour = "Layers") +
  theme_classic()
}

# Define the directory where you want to save the plots
save_dir <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/"

for (i in seq_along(plots)) {
  # Define the file name based on the sample_id for uniqueness, with the full path
  file_name <- paste0(save_dir, "plots/plot_hvg_pca", spe_list[[i]]$sample_id[1], ".png")
  
  # Save the plot using ggsave
  ggsave(filename = file_name, plot = plots[[i]], width = 8, height = 6)
}


2.6 Get top SVGs

nnsvg
The results are stored in the rowData of the SpatialExperiment object.
The main results of interest are:
LR_stat: likelihood ratio (LR) statistics used to rank SVGs
rank: rank of top SVGs according to LR statistics
pval: approximate p-values
padj: approximate p-values adjusted for multiple testing
prop_sv: effect size defined as proportion of spatial variance

First load each spe object after having run nnsvg on the cluster

In [ ]:

# List files in the directory
files <- list.files(path = load_all_data, pattern = "spe_sub_nnSVG", full.names = TRUE) #to load hvgs only


names(files) <- samplesheet

# Initialize a list to store each sample
spe_list_nnsvg <- list()

# Loop through each file and process it
for (sample_name in names(files)) {
 # Read the Visium data
  spe <- readRDS(files[sample_name])

  # Store the processed sample in the list
  spe_list_nnsvg[[sample_name]] <- spe 
  
  
  assign(paste0("spe_", sample_name), spe)
  #print progress check
  cat("Loaded:", sample_name, "\n")
}


get top SVGs for each sample using user defined function

In [ ]:
# Loop through each spe object in the list
for (i in seq_along(spe_list_nnsvg)) {

  spe <- spe_list_nnsvg[[i]]
  
  #Assign TRUE or FALSE based on ranking of gene 
  rowData(spe)$svg <- rowData(spe)$rank <=1000
  #save it back to original list
  spe_list_nnsvg[[i]] <- spe
}


Run PCA and UMAP on top SVGs

In [ ]:
set.seed(456)

#initialise list to store hvgs
svg <- vector("list", length(spe_list_nnsvg))

for (i in seq_along(spe_list_nnsvg)) {
    spe <- spe_list_nnsvg[[i]]
    # get the rownames (gene IDs) where svg == TRUE
    svg[[i]] <- rownames(spe)[rowData(spe)$svg == TRUE]
    # run PCA on those genes
    spe <- runPCA(spe, subset_row = svg[[i]])
    # run UMAP on PCA result
    spe <- runUMAP(spe, dimred = "PCA")
    # save updated object
    spe_list_nnsvg[[i]] <- spe
}


Integrate SVGs and HVGs
A recent benchmark paper (Li et al. 2022) showed that integrating HVGs and SVGs to generate a combined set of features can improve downstream clustering performance in STx data. This confirms that SVGs contain additional biologically relevant information that is not captured by HVGs in these datasets. For example, a simple way to combine these features is to concatenate columns of principal components (PCs) calculated on the set of HVGs and the set of SVGs (excluding overlapping HVGs), and then using the combined set of features for further downstream analyses (Li et al. 2022).

Concatnate and then perform the next stages of the workbook tutorial (clustering on  integrated VGs)
first want to deconvolute to cell type to colour umaps by cell identity, compare with mIF and look for spatial patterns of VGs once i have done clustering

In [ ]:
......

Dimensionality reduction
Use PCA to reduce the dimensions of our dataset to assist clustering and UMAP to further reduce the principal components (PCs) in a two-dimensional space and produce better visualisations for the PCA.
runPCA() function runs PCA on a SCE object, and returns an updated version of the single cell object with the PCA result added to the reducedDim slot.